#Crawler

In [35]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

result = pd.DataFrame()
recent_num = 603464
data_num = 100
for i in range(recent_num - data_num + 1, recent_num+1):
  URL = "https://www1.president.go.kr/petitions/"+str(i)

  response = requests.get(URL)
  html = response.text
  soup = BeautifulSoup(html, "html.parser")
  title = soup.find("h3", class_="petitionsView_title")
  count = soup.find("span", class_="counter")

  for content in soup.select("div.petitionsView_write > div.View_write"):
    content

  a = []
  for tag in soup.select("ul.petitionsView_info_list > li"):
    a.append(tag.contents[1])

  if len(a) != 0:
    df1 = pd.DataFrame({ 'start' : [a[1]],                
                           'end' : [a[2]],                     
                           'category' :  [a[0]],               
                           'count' : [count.text],             
                           'title': [title.text],              
                           'content': [content.text.strip()[0:13000]]                              
                         })
    
    result=pd.concat([result, df1])
    result.index = np.arange(len(result))

    if i % 60 == 0:                                        
        print("Sleep 90seconds. Count:" + str(i)           
              +",  Local Time:"+ time.strftime('%Y-%m-%d', time.localtime(time.time()))
              +" "+ time.strftime('%X', time.localtime(time.time()))
              +",  Data Length:"+ str(len(result)))        
        time.sleep(90) 


In [36]:
df = result
df.head()

""


In [31]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

AttributeError: ignored

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [28]:
df.head()

,start,end,category,count,title,content
0,2020-10-15,2020-11-14,교통/건축/국토,"12,139",노선 신설은 혜택을 받지못하는 23 소외지역 및 동서축 신설이란 신설 목...,요즘 노선 신설을 두고 각 지자체가 지역 국회의원 시도의원 지역언론사 등을 동...
1,2020-10-15,2020-11-14,보건복지,419,10년을 일하고 8년이상 고용보험을 납부했는데 근로자가 아니라서 실업급여를 못준답니다,저는 소규모 섬유공장을 10여년 운영하다가 이번 코로나19 사태로 공장문을 닫을수밖...
2,2020-10-16,2020-11-15,인권/성평등,"6,344",부산 여중 성추행사건 은폐,안녕하십니까 저희는 부산의 여자중학교 의 재학생 졸업생들입니다 최근 뉴스와...
3,2020-10-16,2020-11-15,정치개혁,"1,863",갑자기 언론전담재판부에 복귀한 강민구 삼성장충기문자부장판사 를 해임하라,토착왜구언론과 기레기들 가짜기사때문에 독일의 소녀상을 철거할 빌미를 제공하게 됬다는...
4,2020-10-16,2020-11-15,기타,574,나라가 못지킨 연평도 공무원 북한에게 살해당한 그분의 아이들만이라도 지켜주세요,먼저 두 줄 요약 먼저 하겠습니다 북한의 흉탄에 살해당한 한 가정의 아버지 ...


#Tokenizing

In [9]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [10]:
from konlpy.tag import Okt

okt = Okt()

df["title_token"] = df.title.apply(okt.morphs)
df["content_token"] = df.content.apply(okt.nouns)

In [11]:
df.head()

,start,end,category,count,title,content,title_token,content_token
0,2021-12-28,2022-01-27,보건복지,"2,251",아버지께서 부스터샷 맞고 돌아가셨습니다,아버지께서 부스터샷맞고 돌아가셨습니다 아직까지도 황망함과 슬픔에서 벗어나지 ...,"[아버지, 께서, 부스터, 샷, 맞고, 돌아가셨습니다]","[아버지, 부스터, 샷, 황, 슬픔, 못, 제, 이글, 이유, 아버지, 백신, 희생..."
1,2021-12-28,2022-01-27,기타,125,부산 연제구 건물의 임대인의 도덕성 자질과 골프연습장의 골목상권 침해에 대하여,저는 부산 연제구 한창타워 건물에서 골프연습장을 운영하고 있습니다 같은 층 골프...,"[부산, 연제구, 건물, 의, 임대, 인, 의, 도덕성, 자질, 과, 골프, 연습장...","[저, 부산, 연제구, 한창, 타워, 건물, 골프, 연습장, 운영, 층, 골프, 연..."
2,2021-12-28,2022-01-27,보건복지,"3,068",혐오와 차별을 부추기는 백신패스 철회를 요청합니다,저는 백신 미접종자이며 20대 후반이고 취업 준비생이며 여성입니다 저는 지금 백...,"[혐오, 와, 차별, 을, 부추기는, 백신, 패스, 철회, 를, 요청, 합니다]","[저, 백신, 미접, 종자, 후반, 취업, 준비, 생, 여성, 저, 지금, 백신, ..."
3,2021-12-28,2022-01-27,기타,"4,467",더이상 백신 맞지 않으시기를 간곡히 부탁드립니다,유재석 임창정 정우성 손흥민이 확진이 되어도 회사 동료가 확진이 되어도 그...,"[더, 이상, 백신, 맞지, 않으, 시기, 를, 간곡히, 부탁드립니다]","[유재석, 임창정, 정우성, 손흥민, 확진, 회사, 동료, 확진, 그, 죽, 것, ..."
4,2021-12-28,2022-01-27,보건복지,"1,469",너무도 무서운 고통 전 일상으로 돌아가고 싶어요 코로나 백신접종 강요하지 말아주세요,전 47세 기혼 출산 경험 없는 여성입니다 9월 24일 접종 그날 밤부터 전...,"[너무도, 무서운, 고통, 전, 일상, 으로, 돌아가고, 싶어요, 코로나, 백신, ...","[전, 세, 기혼, 출산, 경험, 여성, 접종, 그날, 밤, 전, 접종, 왼쪽, 부..."


In [13]:
df["token_final"] = df["title_token"] + df["content_token"]
df["count"] = df["count"].replace({",":""}, regex=True).apply(lambda x : int(x))
print(df.dtypes)
df["label"] = df["count"].apply(lambda x: "Yes" if x>=1000 else "No")

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
dtype: object


In [14]:
df.head()

,start,end,category,count,title,content,title_token,content_token,token_final,label
0,2021-12-28,2022-01-27,보건복지,2251,아버지께서 부스터샷 맞고 돌아가셨습니다,아버지께서 부스터샷맞고 돌아가셨습니다 아직까지도 황망함과 슬픔에서 벗어나지 ...,"[아버지, 께서, 부스터, 샷, 맞고, 돌아가셨습니다]","[아버지, 부스터, 샷, 황, 슬픔, 못, 제, 이글, 이유, 아버지, 백신, 희생...","[아버지, 께서, 부스터, 샷, 맞고, 돌아가셨습니다, 아버지, 부스터, 샷, 황,...",Yes
1,2021-12-28,2022-01-27,기타,125,부산 연제구 건물의 임대인의 도덕성 자질과 골프연습장의 골목상권 침해에 대하여,저는 부산 연제구 한창타워 건물에서 골프연습장을 운영하고 있습니다 같은 층 골프...,"[부산, 연제구, 건물, 의, 임대, 인, 의, 도덕성, 자질, 과, 골프, 연습장...","[저, 부산, 연제구, 한창, 타워, 건물, 골프, 연습장, 운영, 층, 골프, 연...","[부산, 연제구, 건물, 의, 임대, 인, 의, 도덕성, 자질, 과, 골프, 연습장...",No
2,2021-12-28,2022-01-27,보건복지,3068,혐오와 차별을 부추기는 백신패스 철회를 요청합니다,저는 백신 미접종자이며 20대 후반이고 취업 준비생이며 여성입니다 저는 지금 백...,"[혐오, 와, 차별, 을, 부추기는, 백신, 패스, 철회, 를, 요청, 합니다]","[저, 백신, 미접, 종자, 후반, 취업, 준비, 생, 여성, 저, 지금, 백신, ...","[혐오, 와, 차별, 을, 부추기는, 백신, 패스, 철회, 를, 요청, 합니다, 저...",Yes
3,2021-12-28,2022-01-27,기타,4467,더이상 백신 맞지 않으시기를 간곡히 부탁드립니다,유재석 임창정 정우성 손흥민이 확진이 되어도 회사 동료가 확진이 되어도 그...,"[더, 이상, 백신, 맞지, 않으, 시기, 를, 간곡히, 부탁드립니다]","[유재석, 임창정, 정우성, 손흥민, 확진, 회사, 동료, 확진, 그, 죽, 것, ...","[더, 이상, 백신, 맞지, 않으, 시기, 를, 간곡히, 부탁드립니다, 유재석, 임...",Yes
4,2021-12-28,2022-01-27,보건복지,1469,너무도 무서운 고통 전 일상으로 돌아가고 싶어요 코로나 백신접종 강요하지 말아주세요,전 47세 기혼 출산 경험 없는 여성입니다 9월 24일 접종 그날 밤부터 전...,"[너무도, 무서운, 고통, 전, 일상, 으로, 돌아가고, 싶어요, 코로나, 백신, ...","[전, 세, 기혼, 출산, 경험, 여성, 접종, 그날, 밤, 전, 접종, 왼쪽, 부...","[너무도, 무서운, 고통, 전, 일상, 으로, 돌아가고, 싶어요, 코로나, 백신, ...",Yes


In [15]:
df_drop = df[["token_final", "label"]]
df_drop.head()

,token_final,label
0,"[아버지, 께서, 부스터, 샷, 맞고, 돌아가셨습니다, 아버지, 부스터, 샷, 황,...",Yes
1,"[부산, 연제구, 건물, 의, 임대, 인, 의, 도덕성, 자질, 과, 골프, 연습장...",No
2,"[혐오, 와, 차별, 을, 부추기는, 백신, 패스, 철회, 를, 요청, 합니다, 저...",Yes
3,"[더, 이상, 백신, 맞지, 않으, 시기, 를, 간곡히, 부탁드립니다, 유재석, 임...",Yes
4,"[너무도, 무서운, 고통, 전, 일상, 으로, 돌아가고, 싶어요, 코로나, 백신, ...",Yes


#Word Embedding

In [18]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop["token_final"],
                           sg=1,
                           size=100,
                           window=2,
                           min_count=1,
                           workers=4
                           )

print(embedding_model)

model_result = embedding_model.wv.most_similar("코로나")
print(model_result)

Word2Vec(vocab=5140, size=100, alpha=0.025)
[('병원', 0.9995991587638855), ('저', 0.9995766878128052), ('수', 0.9995741248130798), ('때', 0.9995672702789307), ('위해', 0.9995507597923279), ('치료', 0.9995436072349548), ('사람', 0.9995393753051758), ('이', 0.9995346069335938), ('시간', 0.9995304346084595), ('것', 0.9995227456092834)]


# 실험 설계

In [ ]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

# tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
# val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

In [ ]:
import torchtext
from torchtext.data import Field

def tokenizer(text):
  text = re.sub('[\[\]\']', '', str(text))
  text = text.split(", ")
  return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential=False)


In [ ]:
from torchtext.data import TabularDataset

# train, validation = TabularDataset.splits(
#     path = 'data/',
#     train = 'train.csv',
#     validation = 'validation.csv',
#     format = 'csv',
#     fields = [('text', TEXT), ('label', LABEL)],
#     skip_header = True
# )

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

In [ ]:
import torch
from torchtext.vocab import Vectors
from torchtext.data import BucketIterator

vectors = Vectors(name="data/petitions_tokens_w2v")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

# Model

In [ ]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

In [ ]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

In [ ]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

In [ ]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
 
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
        
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')